In [18]:
# BOston 부동산 가격 Dataset 호출

from keras.datasets import boston_housing
(train_data,train_labels),(test_data,test_labels) = boston_housing.load_data()

In [19]:
# 호출한 Dataset Shape 확인

print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [20]:
print(train_data[0])
# Data의 차이가 서로 큼 -> 영향이 너무 커진다.
# Scaling 필요함.
# column 총 13개

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [21]:
# Data Preprocessing - Standard Scaling
# by 수동

mean = train_data.mean(axis=0)
# 0번 축을 기준으로 각 column 별로 평균치를 추출
print(mean)

train_data -= mean
# train_data = train_data - mean 과 같은 표헌
# Broadcating 개념 : 차원이 다르더라도 계산이 된다 -> train_data에서 평균치를 다 뺼 수 있다.

std = train_data.std(axis=0)
# train_data의 표준 편차를 구하기
# axis=0 는 모든 column을 지정해주는 라인.

train_data /= std
# train_data를 표준 편차로 나눔

train_data[0]

[3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]


array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [22]:
# Model의 구성

from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    # Regression용 Sequnential 모델 사용

    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    # 회기 값은 그대로 나오면 되기 때문에 activation 지정 안해도 괜찮다.

    model.compile(optimizer='rmsprop',
                loss=['mse'],
                metrics=['mse'])
    return model

# 모델을 여러 번 사용하기 위해서 bulid_model 이란 함수로 정의!
# K-Folder 검증을 위해서 여러번 사용해야함

In [23]:
# K-folder 검증을 사용한 훈련 검증

import numpy as np

k = 4 # folder 개수
num_val_samples = len(train_data)//4 # 한 Folder에 들어가는 train_data 수
all_scores = [] 

for i in range(k):  # 총 4번 반복한다
    print('처리중인 폴드 #', i)
 
    # 검증 데이터의 준비
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_labels = train_labels[i*num_val_samples:(i+1)*num_val_samples]
    
    # 학습 데이터의 준비
    data1 = train_data[:i*num_val_samples]
    data2 = train_data[(i+1)*num_val_samples:]
    data1_labels = train_labels[:i*num_val_samples]
    data2_labels = train_labels[(i+1)*num_val_samples:]

    # 나눈 데이터 다시 합친다
    partial_train_data = np.concatenate([data1,data2],axis=0)
    partial_train_labels = np.concatenate([data1_labels,data2_labels],axis=0)
    
    # 모델의 학습
    model = build_model()
    model.summary()
    
    model.fit(partial_train_data,
              partial_train_labels,
              epochs=500,
              batch_size=128,
              verbose=0) # 훈련상황 보여주지 않음
    
    # 모델의 검증
    val_mse, val_mae = model.evaluate(val_data,val_labels)
    print(val_mse,val_mae)
    all_scores.append(val_mae)

처리중인 폴드 # 0
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                896       
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 2ms/step - loss: 6.8176 - mse: 6.8176
6.817623138427734 6.817623138427734
처리중인 폴드 # 1
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)     

In [24]:
# parameter : 모델의 복잡도를 보여줌
# 같은 성능일 때 parameter가 낮은게 부하가 적다

np.mean(all_scores)

11.117838621139526